In [4]:
from typing import List

import pandas as pd
from IPython.display import display, HTML
from dotenv import load_dotenv
import json

import pymupdf4llm # pymupdf4llm must to be added to the requirememts
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

import system_prompts

load_dotenv()

True

In [5]:
# Define model and output parser
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = JsonOutputParser()

# Extract PDF text as Markdown
doc_path = "../data/2025/Decision/OROURKE_06-05-25_7891__KING_AND_GEORGE_LLC (DECISION).pdf"
md_text = pymupdf4llm.to_markdown(doc_path)

# Create chat prompt teamplates
fact_pattern_exctraction_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            system_prompts.FACT_PATTERN_EXTRACTION_PROMPT
        ),
        ("user", "{user_input}"),
    ]
)

applicable_rules_of_evidence_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            system_prompts.APPICLABLE_RULES_OF_EVIDENCE_EXTRACTION_PROMPT
        ),
        ("user", "{user_input}"),
    ]
)

substantive_rules_exctraction_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            system_prompts.SUBSTANTIVE_RULES_EXTRACTION_PROMPT
        ),
        ("user", "{user_input}"),
    ]
)

substantive_rules_with_pdf_exctraction_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            system_prompts.SUBSTANTIVE_RULES_WITH_PDF_EXTRACTION_PROMPT
        ),
        ("user", "{user_input}"),
    ]
)


# Create the chains
fact_pattern_exctraction_chain = fact_pattern_exctraction_prompt | model | output_parser
applicable_rules_of_evidence_exctraction_chain = applicable_rules_of_evidence_prompt | model | output_parser
susbstantive_rules_exctraction_chain = substantive_rules_exctraction_prompt | model | output_parser
substantive_rules_with_pdf_exctraction_chain = substantive_rules_with_pdf_exctraction_prompt | model | output_parser

### Extract the Fact Pattern from the Court Desicion PDF

In [13]:
doc_path = "data/2025/Decision/OROURKE_06-05-25_7891__KING_AND_GEORGE_LLC (DECISION).pdf"
md_text = pymupdf4llm.to_markdown(doc_path)

fact_pattern_exctraction_input_params = {"user_input": md_text}
fact_pattern_exctraction_response = fact_pattern_exctraction_chain.invoke(fact_pattern_exctraction_input_params)

print(fact_pattern_exctraction_response)

['In January 2018, GSA issued a solicitation for facilities operations and maintenance services for nine federal buildings in Florida.', 'The solicitation sought proposals for a firm-fixed-price contract from small businesses under the 8(a) program and established a period of performance of one base year and three option years.', 'On March 3, 2021, K&G submitted an offer in response to the solicitation.', 'GSA awarded the contract, which incorporated the solicitation terms discussed above, to K&G on April 20, 2021.', 'Despite not being fully staffed consistent with the minimum required staffing levels at all facilities, K&G began performing the base year of the contract on June 1, 2021.', 'The parties used a spreadsheet (the FTE tracker) to identify and track vacant positions.', "On July 8, 2021, the agency’s building management specialist emailed K&G’s project manager stating, 'After discussing the situation with the acquisition team, including the Contracting Officer, it has been dec

### Extract the Applicable Rules Of Evidence from the Fact Pattern

In [15]:
applicable_rules_of_evidence_exctraction_input_params = { "user_input": fact_pattern_exctraction_response}
applicable_rules_of_evidence_exctraction_response = applicable_rules_of_evidence_exctraction_chain.invoke(
    applicable_rules_of_evidence_exctraction_input_params
)

print(applicable_rules_of_evidence_exctraction_response)

[{'rule_number': 'FRE 401', 'rule_title': 'Test for Relevant Evidence', 'issue': 'Relevance of evidence regarding staffing levels and deductions.', 'application': "The evidence regarding K&G's staffing levels and the deductions assessed by GSA is relevant to determine whether K&G fulfilled its contractual obligations.", 'arguments_for_admissibility': "The evidence directly relates to K&G's performance under the contract and the basis for the deductions, which is central to the dispute.", 'arguments_against_admissibility': 'There may be arguments that the evidence is overly prejudicial or that it does not directly pertain to the specific claims being made.', 'likely_admissibility': 'admissible', 'triggering_facts': [4, 6, 7, 8, 9, 10, 11]}, {'rule_number': 'FRE 402', 'rule_title': 'General Admissibility of Relevant Evidence', 'issue': 'General admissibility of evidence related to contract performance.', 'application': 'All relevant evidence is generally admissible unless otherwise exclu

### Extract the Substantive Rules from the Fact Pattern

In [19]:
substantive_rules_exctraction_input_params = { "user_input": fact_pattern_exctraction_response}
substantive_rules_exctraction_response = susbstantive_rules_exctraction_chain.invoke(
    substantive_rules_exctraction_input_params
)

print(substantive_rules_exctraction_response)

[{'rule_label': 'Firm-Fixed-Price Contract Principles', 'rule_source': 'FAR 16.202-1', 'legal_issue': 'Whether deductions for staffing vacancies are permissible under a firm-fixed-price contract.', 'rule_summary': 'Under a firm-fixed-price contract, the contractor is responsible for delivering the agreed-upon services regardless of the costs incurred.', 'application': 'K&G, as the contractor, is obligated to meet the staffing requirements outlined in the contract. The deductions made by GSA for staffing vacancies are consistent with the principle that the contractor bears the risk of performance.', 'supporting_arguments': ['The contract explicitly requires K&G to maintain minimum staffing levels.', 'Deductions for unstaffed positions are a reasonable enforcement of contract terms.'], 'counter_arguments': ['K&G may argue that the deductions are punitive rather than reflective of actual performance.', 'K&G could contend that the contract did not specify the consequences of staffing short

### Extract the Substantive Rules from the Fact Pattern+Court Decision PDF Content

In [23]:
substantive_rules_with_pdf_exctraction_input_params = {
    "pdf_content": md_text,
    "user_input": fact_pattern_exctraction_response
}
substantive_rules_with_pdf_exctraction_response = substantive_rules_with_pdf_exctraction_chain.invoke(
    substantive_rules_with_pdf_exctraction_input_params
)

print(substantive_rules_with_pdf_exctraction_response)

[{'rule_label': 'Firm-Fixed-Price Contract Principles', 'rule_source': 'FAR 52.216-4', 'legal_issue': 'Whether deductions for staffing vacancies are permissible under a firm-fixed-price contract.', 'rule_summary': 'Under a firm-fixed-price contract, the contractor is responsible for delivering the agreed-upon services regardless of the costs incurred, and deductions may only be made if explicitly allowed by the contract terms.', 'application': "The GSA's deductions for staffing vacancies were assessed based on the terms of the contract, which required K&G to maintain minimum staffing levels. The deductions were applied despite K&G's objections, indicating a potential conflict with the contract's fixed-price nature.", 'supporting_arguments': ['The contract explicitly required K&G to maintain certain staffing levels.', 'The GSA communicated the deduction policy clearly to K&G, indicating that deductions would be taken for unstaffed positions.'], 'counter_arguments': ['K&G may argue that 

### Visualization

In [27]:
# Fact Pattern
for i, fact in enumerate(fact_pattern_exctraction_response):
    print(f"{i}. {fact}")

0. In January 2018, GSA issued a solicitation for facilities operations and maintenance services for nine federal buildings in Florida.
1. The solicitation sought proposals for a firm-fixed-price contract from small businesses under the 8(a) program and established a period of performance of one base year and three option years.
2. On March 3, 2021, K&G submitted an offer in response to the solicitation.
3. GSA awarded the contract, which incorporated the solicitation terms discussed above, to K&G on April 20, 2021.
4. Despite not being fully staffed consistent with the minimum required staffing levels at all facilities, K&G began performing the base year of the contract on June 1, 2021.
5. The parties used a spreadsheet (the FTE tracker) to identify and track vacant positions.
6. On July 8, 2021, the agency’s building management specialist emailed K&G’s project manager stating, 'After discussing the situation with the acquisition team, including the Contracting Officer, it has been de

In [ ]:
# Applicable Rules of Evidence
applicable_rules_of_evidence = pd.DataFrame(applicable_rules_of_evidence_exctraction_response)
applicable_rules_of_evidence

,rule_number,rule_title,issue,application,arguments_for_admissibility,arguments_against_admissibility,likely_admissibility,triggering_facts
0,FRE 401,Test for Relevant Evidence,Relevance of evidence regarding staffing level...,The evidence regarding K&G's staffing levels a...,The evidence directly relates to K&G's perform...,There may be arguments that the evidence is ov...,admissible,"[4, 6, 7, 8, 9, 10, 11]"
1,FRE 402,General Admissibility of Relevant Evidence,General admissibility of evidence related to c...,All relevant evidence is generally admissible ...,The evidence is relevant to the issues at hand...,Potential arguments could be made regarding th...,admissible,"[4, 6, 7, 8, 9, 10, 11]"
2,FRE 403,Exclusion of Relevant Evidence on Grounds of P...,Potential for evidence to cause unfair prejudi...,"While the evidence is relevant, there may be c...",The probative value of the evidence outweighs ...,The evidence may distract from the main issues...,depends,"[4, 6, 7, 8, 9, 10, 11]"
3,FRE 801,Definitions That Apply to This Article; Exclus...,Hearsay implications of communications between...,The email from the building management special...,The email may be admissible as a statement of ...,If offered to prove the truth of the deduction...,depends,[6]
4,FRE 803,Exceptions to the Rule Against Hearsay,Exceptions that may apply to hearsay statements.,Certain statements made in the course of busin...,The email may qualify as a business record or ...,The context of the statement may not meet the ...,depends,[6]


In [28]:
# Applicable Substantive Rules
substantive_rules = pd.DataFrame(substantive_rules_with_pdf_exctraction_response)
substantive_rules

,rule_label,rule_source,legal_issue,rule_summary,application,supporting_arguments,counter_arguments,likely_outcome,triggering_facts
0,Firm-Fixed-Price Contract Principles,FAR 52.216-4,Whether deductions for staffing vacancies are ...,"Under a firm-fixed-price contract, the contrac...",The GSA's deductions for staffing vacancies we...,[The contract explicitly required K&G to maint...,[K&G may argue that the deductions were not ex...,uncertain,"[4, 6, 7, 8, 10]"
1,Modification of Contracts,Contract Disputes Act,Whether the unilateral modification by GSA to ...,A contracting officer may unilaterally modify ...,GSA's unilateral modification to deobligate fu...,[The Contracting Officer has the authority to ...,"[K&G did not agree to the modifications, which...",uncertain,"[9, 10, 11]"
2,Notice and Opportunity to Cure,Common Law Principle,Whether K&G was given adequate notice and oppo...,Contractors are generally entitled to notice o...,K&G received notice of the deductions after th...,[The email from GSA indicated a decision had b...,[GSA may argue that the contract terms allowed...,applies,"[6, 8, 10]"
